In [ ]:
# ! pip install gensim

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import nltk
import re
from bs4 import BeautifulSoup
import os
import contractions

# os.chdir('/content/drive/Shared drives/USC_CSCI544-Applied NLP/HWs/HW2') # where the files for this project are

C:\Users\navsa\AppData\Local\Temp\ipykernel_16584\3298560112.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import Perceptron, LogisticRegression
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import gensim
import gensim.downloader as api

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset


In [ ]:
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\navsa\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\navsa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\navsa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\navsa\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\navsa\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

# **READ DATA**

In [ ]:
df=pd.read_table('amazon_reviews_us_Office_Products_v1_00.tsv', on_bad_lines='skip')

df = df[['review_body', 'star_rating']]


df['star_rating'] = pd.to_numeric(df['star_rating'], errors='coerce')

df = df.dropna(subset=['review_body'], how='all')

print(df)


C:\Users\navsa\AppData\Local\Temp\ipykernel_16584\1375649650.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_table('amazon_reviews_us_Office_Products_v1_00.tsv', on_bad_lines='skip')


                                               review_body  star_rating
0                                           Great product.          5.0
1        What's to say about this commodity item except...          5.0
2          Haven't used yet, but I am sure I will like it.          5.0
3        Although this was labeled as &#34;new&#34; the...          1.0
4                          Gorgeous colors and easy to use          4.0
...                                                    ...          ...
2640249  I can't live anymore whithout my Palm III. But...          4.0
2640250  Although the Palm Pilot is thin and compact it...          4.0
2640251  This book had a lot of great content without b...          4.0
2640252  I am teaching a course in Excel and am using t...          5.0
2640253  A very comprehensive layout of exactly how Vis...          5.0

[2640080 rows x 2 columns]


# **CREATE BALANCED DATASET**

In [ ]:
def create_balanced_dataset(df, samples_per_rating=50000):
    balanced_df = pd.DataFrame()
    for rating in range(1, 6):
        rating_df = df[df['star_rating'] == rating].sample(samples_per_rating)
        balanced_df = pd.concat([balanced_df, rating_df], ignore_index=True)
    return balanced_df

df = create_balanced_dataset(df)

print(df)

                                              review_body  star_rating
0       It was good at first. I loved how it could pri...          1.0
1       Did not work on my Samsung ML-2955ND. I will h...          1.0
2       I installed and uninstalled over and over and ...          1.0
3       Sometimes, you learn the hard way.<br /><br />...          1.0
4       I bought this in 2011 of october and as of Aug...          1.0
...                                                   ...          ...
249995  I bought these originally to hold overflow tra...          5.0
249996  Very good buy so far.  This is the first time ...          5.0
249997  This is a really good looking calendar. The ph...          5.0
249998  These are super cool! When I was a kid, we use...          5.0
249999  THIS PRODUCT IS AWESOME! PERFECT FOR YOUR CAR ...          5.0

[250000 rows x 2 columns]


# **CREATE TERNARY LABELS**

In [ ]:
df['sentiment'] = np.where(df['star_rating'] <= 2, 2,  # Negative: 2
                          np.where(df['star_rating'] > 3, 1, 3))  # Positive: 1, Neutral: 3

# df['sentiment'] = np.where(df['star_rating'] > 3, 1, 0)  # Positive: 1, Negative: 0

# Print review counts per class
print("Number of positive reviews:", df[df['sentiment'] == 1].shape[0])
print("Number of negative reviews:", df[df['sentiment'] == 2].shape[0])
print("Number of neutral reviews:", df[df['sentiment'] == 3].shape[0])

print(df)

Number of positive reviews: 100000
Number of negative reviews: 100000
Number of neutral reviews: 50000
                                              review_body  star_rating  \
0       It was good at first. I loved how it could pri...          1.0   
1       Did not work on my Samsung ML-2955ND. I will h...          1.0   
2       I installed and uninstalled over and over and ...          1.0   
3       Sometimes, you learn the hard way.<br /><br />...          1.0   
4       I bought this in 2011 of october and as of Aug...          1.0   
...                                                   ...          ...   
249995  I bought these originally to hold overflow tra...          5.0   
249996  Very good buy so far.  This is the first time ...          5.0   
249997  This is a really good looking calendar. The ph...          5.0   
249998  These are super cool! When I was a kid, we use...          5.0   
249999  THIS PRODUCT IS AWESOME! PERFECT FOR YOUR CAR ...          5.0   

        

# **CREATE BINARY LABELS**

In [ ]:
# Data cleaning and preprocessing
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# Data Cleaning
def clean_text(text):
    # Remove HTML tags
    if isinstance(text, str) and text:
      soup = BeautifulSoup(text, 'html.parser')
      text = soup.get_text()

      # Expand contractions
      text = contractions.fix(text)

      # Remove special characters and digits
      text = re.sub(r'[^a-zA-Z\s]', '', text)

      # Remove URLs
      text = re.sub(r'https?://\S+', '', text)

      # Convert to lowercase
      text = text.lower()

      # Tokenize
      words = nltk.word_tokenize(text)

      # Remove stop words
      stop_words = set(nltk.corpus.stopwords.words('english'))
      words = [word for word in words if word not in stop_words]

      # Lemmatize
      lemmatizer = WordNetLemmatizer()
      words = [lemmatizer.lemmatize(word) for word in words]

      # Remove extra spaces
      text = ' '.join(words)
      text = re.sub(r'\s+', ' ', text).strip()  # Replace multiple spaces with single space

      return text
    else:
      return ''  # Return an empty string for empty or non-string inputs

df['clean_review'] = df['review_body'].apply(clean_text)
# Print the average length of the cleaned reviews
print(df)
print("Average length of cleaned reviews(TERTIARY):", df['clean_review'].str.len().mean())


C:\Users\navsa\AppData\Local\Temp\ipykernel_16584\668557797.py:9: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, 'html.parser')
C:\Users\navsa\AppData\Local\Temp\ipykernel_16584\668557797.py:9: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  soup = BeautifulSoup(text, 'html.parser')


                                              review_body  star_rating  \
0       It was good at first. I loved how it could pri...          1.0   
1       Did not work on my Samsung ML-2955ND. I will h...          1.0   
2       I installed and uninstalled over and over and ...          1.0   
3       Sometimes, you learn the hard way.<br /><br />...          1.0   
4       I bought this in 2011 of october and as of Aug...          1.0   
...                                                   ...          ...   
249995  I bought these originally to hold overflow tra...          5.0   
249996  Very good buy so far.  This is the first time ...          5.0   
249997  This is a really good looking calendar. The ph...          5.0   
249998  These are super cool! When I was a kid, we use...          5.0   
249999  THIS PRODUCT IS AWESOME! PERFECT FOR YOUR CAR ...          5.0   

        sentiment                                       clean_review  
0               2  good first loved coul

In [ ]:

df_binary = df[df['star_rating'] != 3]  # Discard neutral reviews (rating 3)

# Print review counts per class
print("Number of positive reviews:", df_binary[df_binary['sentiment'] == 1].shape[0])
print("Number of negative reviews:", df_binary[df_binary['sentiment'] == 2].shape[0])
print("Number of neutral reviews (discarded):", len(df_binary[df_binary['star_rating'] == 3]))
print(df_binary)
print("Average length of cleaned reviews(BINARY):", df_binary['clean_review'].str.len().mean())


Number of positive reviews: 100000
Number of negative reviews: 100000
Number of neutral reviews (discarded): 0
                                              review_body  star_rating  \
0       It was good at first. I loved how it could pri...          1.0   
1       Did not work on my Samsung ML-2955ND. I will h...          1.0   
2       I installed and uninstalled over and over and ...          1.0   
3       Sometimes, you learn the hard way.<br /><br />...          1.0   
4       I bought this in 2011 of october and as of Aug...          1.0   
...                                                   ...          ...   
249995  I bought these originally to hold overflow tra...          5.0   
249996  Very good buy so far.  This is the first time ...          5.0   
249997  This is a really good looking calendar. The ph...          5.0   
249998  These are super cool! When I was a kid, we use...          5.0   
249999  THIS PRODUCT IS AWESOME! PERFECT FOR YOUR CAR ...          5.0   



# **SPLIT DATASET**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df['review_body'], df['sentiment'], test_size=0.2, random_state=42)
X_train_binary, X_test_binary, y_train_binary, y_test_binary = train_test_split(df_binary['review_body'], df_binary['sentiment'], test_size=0.2, random_state=42)


# **WORD EMBEDDING**

In [ ]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [ ]:
# Load pretrained model
pretrained_model = api.load('word2vec-google-news-300')


2024-02-08 07:04:27,480 : INFO : loading projection weights from C:\Users\navsa/gensim-data\word2vec-google-news-300\word2vec-google-news-300.gz


2024-02-08 07:05:23,835 : INFO : KeyedVectors lifecycle event {'msg': 'loaded (3000000, 300) matrix of type float32 from C:\\Users\\navsa/gensim-data\\word2vec-google-news-300\\word2vec-google-news-300.gz', 'binary': True, 'encoding': 'utf8', 'datetime': '2024-02-08T07:05:23.833849', 'gensim': '4.3.2', 'python': '3.12.0 (tags/v3.12.0:0fb18b0, Oct  2 2023, 13:03:39) [MSC v.1935 64 bit (AMD64)]', 'platform': 'Windows-11-10.0.22621-SP0', 'event': 'load_word2vec_format'}


In [ ]:
print("PRETRAINED WORD2VEC")
king_minus_man_plus_woman = pretrained_model.most_similar(positive=['king', 'woman'], negative=['man', 'men'], topn=1)
excellent_similar_outstanding = pretrained_model.similarity('excellent', 'outstanding')

print("Semantic similarity between 'king - man + woman' and 'queen':", king_minus_man_plus_woman)
print("Semantic similarity between 'excellent' and 'outstanding':", excellent_similar_outstanding)

PRETRAINED WORD2VEC
Semantic similarity between 'king - man + woman' and 'queen': [('queen', 0.46892058849334717)]
Semantic similarity between 'excellent' and 'outstanding': 0.55674857


In [ ]:
# Tokenize the reviews for training the Word2Vec model
tokenized_reviews = [nltk.word_tokenize(review) for review in X_train]
tokenized_reviews_binary = [nltk.word_tokenize(review) for review in X_train_binary]

# Train a Word2Vec model using the tokenized reviews
word2vec_model = gensim.models.Word2Vec(sentences=tokenized_reviews, vector_size=300, window=11, min_count=10)
word2vec_model_binary = gensim.models.Word2Vec(sentences=tokenized_reviews_binary, vector_size=300, window=11, min_count=10)


2024-02-08 07:09:17,610 : INFO : collecting all words and their counts
2024-02-08 07:09:17,611 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2024-02-08 07:09:17,914 : INFO : PROGRESS: at sentence #10000, processed 732015 words, keeping 26253 word types
2024-02-08 07:09:18,155 : INFO : PROGRESS: at sentence #20000, processed 1470902 words, keeping 38849 word types
2024-02-08 07:09:18,474 : INFO : PROGRESS: at sentence #30000, processed 2247079 words, keeping 49956 word types
2024-02-08 07:09:18,711 : INFO : PROGRESS: at sentence #40000, processed 2996889 words, keeping 58828 word types
2024-02-08 07:09:18,992 : INFO : PROGRESS: at sentence #50000, processed 3762337 words, keeping 66976 word types
2024-02-08 07:09:19,213 : INFO : PROGRESS: at sentence #60000, processed 4495574 words, keeping 74047 word types
2024-02-08 07:09:19,506 : INFO : PROGRESS: at sentence #70000, processed 5250328 words, keeping 80859 word types
2024-02-08 07:09:19,859 : INFO : PROGRES

In [ ]:
print("TRAINED WORD2VEC")

# Demonstrate semantic similarities for the trained Word2Vec model
king_minus_man_plus_woman_custom = word2vec_model.wv.most_similar(positive=['king', 'woman'], negative=['man'], topn=1)
excellent_similar_outstanding_custom = word2vec_model.wv.similarity('excellent', 'outstanding')

print("Semantic similarity between 'king - man + woman' and 'queen' (custom):", king_minus_man_plus_woman_custom)
print("Semantic similarity between 'excellent' and 'outstanding' (custom):", excellent_similar_outstanding_custom)


TRAINED WORD2VEC
Semantic similarity between 'king - man + woman' and 'queen' (custom): [('Binders', 0.471794992685318)]
Semantic similarity between 'excellent' and 'outstanding' (custom): 0.869806


# **SIMPLE MODELS**


In [ ]:
# Define a function to represent each example with Word2Vec features
def get_word2vec_features(review_text):
    feature_vector = np.zeros((word2vec_model.vector_size,), dtype="float32")
    num_words = 0
    for word in review_text:
        if word in word2vec_model.wv:
            feature_vector = np.add(feature_vector, word2vec_model.wv[word])
            num_words += 1
    if num_words != 0:
        feature_vector = np.divide(feature_vector, num_words)
    return feature_vector

# Represent each example with Word2Vec features
X_train_word2vec = np.array([get_word2vec_features(review) for review in tokenized_reviews])
print("X_train:")
print(X_train)
print('X_train_word2vec:')
print(X_train_word2vec)

X_train_word2vec_binary = np.array([get_word2vec_features(review) for review in tokenized_reviews_binary])
print("X_train_binary:")
print(X_train_binary)
print('X_train_word2vec:')
print(X_train_word2vec_binary)





X_train:
75381     not good... after printing 10 pages the black ...
65569                                          out of date.
163473    Ok phone for the price. It did what I needed i...
90518     I just set up my printer.  However, while gett...
138866    It is super cute, but I am not sure of its las...
                                ...                        
119879    Easy to install unfortunately for me I would n...
103694    the negatives first:<br />- Big ink eater.<br ...
131932    This product only lasts one month once you ope...
146867    The ink is on the dry side.  All in all, it is...
121958    I got the inks in a timely manner however ever...
Name: review_body, Length: 200000, dtype: object
X_train_word2vec:
[[ 0.2017009  -0.07907066 -0.5368596  ... -0.02009328  0.18456513
  -0.6514082 ]
 [-0.18326068 -0.27163714 -0.15210694 ...  0.4901157   0.1256411
  -0.591722  ]
 [ 0.2116916  -0.7891235   0.08931109 ...  0.40844712  0.0389001
  -0.28632304]
 ...
 [ 0.38853553 -0.1

In [ ]:
# Tokenize the reviews for training the Word2Vec model
tokenized_reviews_test = [nltk.word_tokenize(review) for review in X_test]
tokenized_reviews_test_binary = [nltk.word_tokenize(review) for review in X_test_binary]

# Represent each example with Word2Vec features
X_test_word2vec = np.array([get_word2vec_features(review) for review in tokenized_reviews_test])
print("X_test:")
print(X_test)
print('X_test_word2vec:')
print(X_test_word2vec)

X_test_word2vec_binary = np.array([get_word2vec_features(review) for review in tokenized_reviews_test_binary])
print("X_test_binary:")
print(X_test_binary)
print('X_test_word2vec:')
print(X_test_word2vec_binary)

X_test:
38683     Product is not what it seems. drivers are all ...
64939     I have no problem with the comfort of this sty...
3954      The ink never worked.  I guess the printer rec...
120374    This is a printer for the more serious photo e...
172861    I ordered these cartridges Nov 2012 despite so...
                                ...                        
179545    Yes, it only does a few pages at a time, but i...
222647    My son loves this lunch box. Plenty of room in...
171823    UPDATE: Just loaded the replacement system on ...
135782    It is a nice pen and it writes pretty smoothly...
208380                                                 fine
Name: review_body, Length: 50000, dtype: object
X_test_word2vec:
[[ 0.2107049  -0.03894419 -0.12380177 ...  0.4152071  -0.01419189
  -0.4963122 ]
 [ 0.2319088   0.07627508  0.02765747 ...  0.40654182  0.00452649
  -0.30042556]
 [ 0.01375213 -0.22278257  0.16280225 ...  0.38947886  0.27099377
  -0.23333038]
 ...
 [ 0.30687958  0.01

In [ ]:
def get_pretrained_features(review_text):
    feature_vector = np.zeros((pretrained_model.vector_size,), dtype="float32")
    num_words = 0
    for word in review_text:
        if word in pretrained_model:
            feature_vector += pretrained_model[word]
            num_words += 1
    if num_words != 0:
        feature_vector /= num_words
    return feature_vector

# Represent each example with Word2Vec features
X_train_pretrained = np.array([get_pretrained_features(review) for review in tokenized_reviews])
print("X_train:")
print(X_train)
print('X_train_pretrained:')
print(X_train_pretrained)

X_train_pretrained_binary = np.array([get_pretrained_features(review) for review in tokenized_reviews_binary])
print("X_train_binary:")
print(X_train_binary)
print('X_train_pretrained_binary:')
print(X_train_pretrained_binary)


X_train:
75381     not good... after printing 10 pages the black ...
65569                                          out of date.
163473    Ok phone for the price. It did what I needed i...
90518     I just set up my printer.  However, while gett...
138866    It is super cute, but I am not sure of its las...
                                ...                        
119879    Easy to install unfortunately for me I would n...
103694    the negatives first:<br />- Big ink eater.<br ...
131932    This product only lasts one month once you ope...
146867    The ink is on the dry side.  All in all, it is...
121958    I got the inks in a timely manner however ever...
Name: review_body, Length: 200000, dtype: object
X_train_pretrained:
[[ 2.00366974e-02  7.23719001e-02 -8.65077972e-03 ...  7.31647015e-03
   2.55513191e-03 -3.31020355e-02]
 [ 6.95800781e-02 -6.98242188e-02  1.13891602e-01 ...  5.51757812e-02
  -1.22802734e-01 -1.23901367e-01]
 [ 4.91333008e-02  3.28812599e-02  6.25712052e-02 ..

In [ ]:
# Represent each example with Pretrained features
X_test_pretrained = np.array([get_pretrained_features(review) for review in tokenized_reviews_test])
print("X_test:")
print(X_test)
print('X_test_word2vec:')
print(X_test_pretrained)

X_test_pretrained_binary = np.array([get_pretrained_features(review) for review in tokenized_reviews_test_binary])
print("X_train_binary:")
print(X_test_binary)
print('X_train_word2vec:')
print(X_test_pretrained_binary)

X_test:
38683     Product is not what it seems. drivers are all ...
64939     I have no problem with the comfort of this sty...
3954      The ink never worked.  I guess the printer rec...
120374    This is a printer for the more serious photo e...
172861    I ordered these cartridges Nov 2012 despite so...
                                ...                        
179545    Yes, it only does a few pages at a time, but i...
222647    My son loves this lunch box. Plenty of room in...
171823    UPDATE: Just loaded the replacement system on ...
135782    It is a nice pen and it writes pretty smoothly...
208380                                                 fine
Name: review_body, Length: 50000, dtype: object
X_test_word2vec:
[[ 0.0372862  -0.00168436  0.05948415 ... -0.04786951  0.01604865
  -0.04362757]
 [ 0.03357273  0.00863713  0.0109748  ... -0.04012519  0.00613607
   0.01166263]
 [ 0.06132334  0.03399953  0.04939803 ... -0.05105566  0.02355414
  -0.06073438]
 ...
 [ 0.02403376  0.04

#### **TDIDF**

In [ ]:
tfidf_vectorizer = TfidfVectorizer()
X = tfidf_vectorizer.fit_transform(df_binary['clean_review'])
y = df_binary['sentiment']

# shape of the feature matrix
print("Shape of feature matrix (X):", X.shape)

# Split the data into training and testing sets
X_train_cleaned, X_test_cleaned, y_train_cleaned, y_test_cleaned = train_test_split(X, y, test_size=0.2, random_state=42)


Shape of feature matrix (X): (200000, 130597)


In [ ]:
perceptron_classifier = Perceptron(max_iter=1000)
perceptron_classifier.fit(X_train_cleaned, y_train_cleaned)
y_pred_perceptron = perceptron_classifier.predict(X_test_cleaned)
print("Perceptron Accuracy (TF-IDF):", accuracy_score(y_test_cleaned, y_pred_perceptron))

Perceptron Accuracy (TF-IDF): 0.816475


In [ ]:
svm_classifier = LinearSVC()
svm_classifier.fit(X_train_cleaned, y_train_cleaned)
y_pred_svm = svm_classifier.predict(X_test_cleaned)
print("SVM Accuracy (TF-IDF):", accuracy_score(y_test_cleaned, y_pred_svm))

c:\Python312\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


SVM Accuracy (TF-IDF): 0.8651


#### **PRE-TRAINED**

In [ ]:
# Train Perceptron and SVM models using pretrained features
perceptron_classifier_pretrained = Perceptron(max_iter=1000)
perceptron_classifier_pretrained.fit(X_train_pretrained_binary, y_train_binary)
y_pred_perceptron_pretrained = perceptron_classifier_pretrained.predict(X_test_pretrained_binary)
print("Perceptron Accuracy (Word2Vec):", accuracy_score(y_test_binary, y_pred_perceptron_pretrained))

svm_classifier_pretrained =  LinearSVC()
svm_classifier_pretrained.fit(X_train_pretrained_binary, y_train_binary)
y_pred_svm_pretrained = svm_classifier_pretrained.predict(X_test_pretrained_binary)
print("SVM Accuracy (Word2Vec):", accuracy_score(y_test_binary, y_pred_svm_pretrained))

Perceptron Accuracy (Word2Vec): 0.7555


c:\Python312\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


SVM Accuracy (Word2Vec): 0.83135


#### **WORD2VEC**

In [ ]:
# Train Perceptron and SVM models using Word2Vec features
perceptron_classifier_word2vec = Perceptron(max_iter=1000)
perceptron_classifier_word2vec.fit(X_train_word2vec_binary, y_train_binary)
y_pred_perceptron_word2vec = perceptron_classifier_word2vec.predict(X_test_word2vec_binary)
print("Perceptron Accuracy (Word2Vec):", accuracy_score(y_test_binary, y_pred_perceptron_word2vec))

svm_classifier_word2vec =  LinearSVC()
svm_classifier_word2vec.fit(X_train_word2vec_binary, y_train_binary)
y_pred_svm_word2vec = svm_classifier_word2vec.predict(X_test_word2vec_binary)
print("SVM Accuracy (Word2Vec):", accuracy_score(y_test_binary, y_pred_svm_word2vec))

Perceptron Accuracy (Word2Vec): 0.7956


c:\Python312\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


SVM Accuracy (Word2Vec): 0.8656


c:\Python312\Lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


# **Feedforward Neural Networks**

In [ ]:

# Define the MLP model
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.fc3 = nn.Linear(hidden_size2, output_size)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return self.softmax(x)

### **AVG METHOD**

##### **WORD2VEC BINARY**

In [ ]:
# Convert the Word2Vec features to PyTorch tensors
X_train_tensor_binary = torch.tensor(X_train_word2vec_binary, dtype=torch.float32)
y_train_tensor_binary = torch.tensor(y_train_binary-1, dtype=torch.long)  # Assuming y_train_binary contains binary labels
X_test_tensor_binary = torch.tensor(X_test_word2vec_binary, dtype=torch.float32)
y_test_tensor_binary = torch.tensor(y_test_binary.values-1, dtype=torch.long)  # Convert Series to numpy array with .values attribute

# Define hyperparameters
input_size = X_train_tensor_binary.shape[1]
hidden_size1 = 50
hidden_size2 = 10
output_size = 2  # Binary classification


In [ ]:
print(y_train_tensor_binary)

tensor([0, 1, 0,  ..., 0, 0, 0])


In [ ]:
# Create DataLoader for training
train_dataset_binary = TensorDataset(X_train_tensor_binary, y_train_tensor_binary)
train_loader_binary = DataLoader(train_dataset_binary, batch_size=64, shuffle=True)

# Initialize the model, loss function, and optimizer
model = MLP(input_size, hidden_size1, hidden_size2, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor_binary)
    loss = criterion(outputs, y_train_tensor_binary)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss}")

Epoch 1, Loss: 0.6967177987098694
Epoch 2, Loss: 0.694940447807312
Epoch 3, Loss: 0.6932215094566345
Epoch 4, Loss: 0.6913893818855286
Epoch 5, Loss: 0.6894699931144714
Epoch 6, Loss: 0.6874707341194153
Epoch 7, Loss: 0.6853677034378052
Epoch 8, Loss: 0.6831188201904297
Epoch 9, Loss: 0.6806842088699341
Epoch 10, Loss: 0.6780391931533813
Epoch 11, Loss: 0.6751617789268494
Epoch 12, Loss: 0.6720513105392456
Epoch 13, Loss: 0.6687220335006714
Epoch 14, Loss: 0.6651893258094788
Epoch 15, Loss: 0.66147381067276
Epoch 16, Loss: 0.6575876474380493
Epoch 17, Loss: 0.6535401344299316
Epoch 18, Loss: 0.6493402719497681
Epoch 19, Loss: 0.6449918746948242
Epoch 20, Loss: 0.6405007839202881
Epoch 21, Loss: 0.6358786225318909
Epoch 22, Loss: 0.6311385631561279
Epoch 23, Loss: 0.6262966990470886
Epoch 24, Loss: 0.6213709712028503
Epoch 25, Loss: 0.6163766980171204
Epoch 26, Loss: 0.6113384962081909
Epoch 27, Loss: 0.6062882542610168
Epoch 28, Loss: 0.6012606620788574
Epoch 29, Loss: 0.59628927707672

In [ ]:
# Evaluate the model on the test set
model.eval()
with torch.no_grad():
    outputs = model(X_test_tensor_binary)
    _, predicted = torch.max(outputs, 1)
    accuracy = (predicted == y_test_tensor_binary).sum().item() / len(y_test_tensor_binary)
    print("Accuracy (WORD2VEC BINARY):", accuracy)

Accuracy (WORD2VEC BINARY): 0.852975


#####  **WORD2VEC TERTIARY**

In [ ]:
# Convert the Word2Vec features to PyTorch tensors
X_train_tensor = torch.tensor(X_train_word2vec, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train-1, dtype=torch.long)  # Assuming y_train_binary contains binary labels
X_test_tensor = torch.tensor(X_test_word2vec, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values-1, dtype=torch.long)  # Convert Series to numpy array with .values attribute

# Define hyperparameters
input_size = X_train_tensor.shape[1]
hidden_size1 = 50
hidden_size2 = 10
output_size = 3  # Tertiary classification

In [ ]:
print(y_train_tensor)

tensor([1, 1, 0,  ..., 2, 2, 2])


In [ ]:
# Create DataLoader for training
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Initialize the model, loss function, and optimizer
model = MLP(input_size, hidden_size1, hidden_size2, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss}")

Epoch 1, Loss: 1.1127711534500122


Epoch 2, Loss: 1.1096194982528687
Epoch 3, Loss: 1.1063947677612305
Epoch 4, Loss: 1.10308837890625
Epoch 5, Loss: 1.0996514558792114
Epoch 6, Loss: 1.0960378646850586
Epoch 7, Loss: 1.0922117233276367
Epoch 8, Loss: 1.0881465673446655
Epoch 9, Loss: 1.083852767944336
Epoch 10, Loss: 1.0793362855911255
Epoch 11, Loss: 1.074615716934204
Epoch 12, Loss: 1.069711446762085
Epoch 13, Loss: 1.0646355152130127
Epoch 14, Loss: 1.0593911409378052
Epoch 15, Loss: 1.0539799928665161
Epoch 16, Loss: 1.048409104347229
Epoch 17, Loss: 1.0427024364471436
Epoch 18, Loss: 1.0369000434875488
Epoch 19, Loss: 1.031054139137268
Epoch 20, Loss: 1.0252203941345215
Epoch 21, Loss: 1.0194531679153442
Epoch 22, Loss: 1.0137931108474731
Epoch 23, Loss: 1.0082675218582153
Epoch 24, Loss: 1.0028938055038452
Epoch 25, Loss: 0.9976795315742493
Epoch 26, Loss: 0.9926290512084961
Epoch 27, Loss: 0.987740695476532
Epoch 28, Loss: 0.9830119609832764
Epoch 29, Loss: 0.9784449934959412
Epoch 30, Loss: 0.9740492105484009
E

In [ ]:
# Evaluate the model on the test set
model.eval()
with torch.no_grad():
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs, 1)
    accuracy = (predicted == y_test_tensor).sum().item() / len(y_test_tensor)
    print("Accuracy (WORD2VEC TERTIARY):", accuracy)

Accuracy (WORD2VEC TERTIARY): 0.67886


#####  **PRE_TRAINED BINARY**

In [ ]:
# Convert the Word2Vec features to PyTorch tensors
X_train_tensor_binary = torch.tensor(X_train_pretrained_binary, dtype=torch.float32)
y_train_tensor_binary = torch.tensor(y_train_binary-1, dtype=torch.long)  # Assuming y_train_binary contains binary labels
X_test_tensor_binary = torch.tensor(X_test_pretrained_binary, dtype=torch.float32)
y_test_tensor_binary = torch.tensor(y_test_binary.values-1, dtype=torch.long)  # Convert Series to numpy array with .values attribute

# Define hyperparameters
input_size = X_train_tensor_binary.shape[1]
hidden_size1 = 50
hidden_size2 = 10
output_size = 2  # Binary classification


In [ ]:
print(y_train_tensor_binary)

tensor([0, 1, 0,  ..., 0, 0, 0])


In [ ]:
# Create DataLoader for training
train_dataset_binary = TensorDataset(X_train_tensor_binary, y_train_tensor_binary)
train_loader_binary = DataLoader(train_dataset_binary, batch_size=64, shuffle=True)

# Initialize the model, loss function, and optimizer
model = MLP(input_size, hidden_size1, hidden_size2, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor_binary)
    loss = criterion(outputs, y_train_tensor_binary)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss}")

Epoch 1, Loss: 0.6963824033737183
Epoch 2, Loss: 0.6959983706474304


Epoch 3, Loss: 0.6956819891929626
Epoch 4, Loss: 0.695370078086853
Epoch 5, Loss: 0.6950533986091614
Epoch 6, Loss: 0.6947672963142395
Epoch 7, Loss: 0.6945233345031738
Epoch 8, Loss: 0.6942496299743652
Epoch 9, Loss: 0.6939220428466797
Epoch 10, Loss: 0.6935703754425049
Epoch 11, Loss: 0.6932117938995361
Epoch 12, Loss: 0.6928437948226929
Epoch 13, Loss: 0.692457914352417
Epoch 14, Loss: 0.6920475363731384
Epoch 15, Loss: 0.6916114687919617
Epoch 16, Loss: 0.6911475658416748
Epoch 17, Loss: 0.6906532645225525
Epoch 18, Loss: 0.6901236772537231
Epoch 19, Loss: 0.689552903175354
Epoch 20, Loss: 0.6889275312423706
Epoch 21, Loss: 0.6882386803627014
Epoch 22, Loss: 0.6874949932098389
Epoch 23, Loss: 0.6866989731788635
Epoch 24, Loss: 0.685836911201477
Epoch 25, Loss: 0.6848797798156738
Epoch 26, Loss: 0.6837737560272217
Epoch 27, Loss: 0.6825416088104248
Epoch 28, Loss: 0.6813512444496155
Epoch 29, Loss: 0.6801514029502869
Epoch 30, Loss: 0.678846538066864
Epoch 31, Loss: 0.67740905284881

In [ ]:
# Evaluate the model on the test set
model.eval()
with torch.no_grad():
    outputs = model(X_test_tensor_binary)
    _, predicted = torch.max(outputs, 1)
    accuracy = (predicted == y_test_tensor_binary).sum().item() / len(y_test_tensor_binary)
    print("Accuracy (WORD2VEC BINARY):", accuracy)

Accuracy (WORD2VEC BINARY): 0.792575


#####  **PRE_TRAINED TERTIARY**

In [ ]:
# Convert the Word2Vec features to PyTorch tensors
X_train_tensor = torch.tensor(X_train_pretrained, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train-1, dtype=torch.long)  # Assuming y_train_binary contains binary labels
X_test_tensor = torch.tensor(X_test_pretrained, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values-1, dtype=torch.long)  # Convert Series to numpy array with .values attribute

# Define hyperparameters
input_size = X_train_tensor.shape[1]
hidden_size1 = 50
hidden_size2 = 10
output_size = 3  # Binary classification

In [ ]:
print(X_train_tensor)
print(y_train_tensor)

tensor([[ 0.0125,  0.0380,  0.0217,  ..., -0.0304,  0.0146, -0.0478],
        [-0.0041,  0.0431,  0.0086,  ..., -0.0256,  0.0155, -0.0352],
        [ 0.0405,  0.0625, -0.0175,  ..., -0.1562,  0.0019, -0.0698],
        ...,
        [ 0.0073,  0.0507,  0.0160,  ..., -0.0472,  0.0499,  0.0199],
        [ 0.0491,  0.0376,  0.0664,  ..., -0.0399,  0.0535, -0.0417],
        [ 0.0251,  0.0570, -0.0098,  ..., -0.0155,  0.0093, -0.0694]])
tensor([1, 1, 0,  ..., 2, 2, 2])


In [ ]:
# Create DataLoader for training
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Initialize the model, loss function, and optimizer
model = MLP(input_size, hidden_size1, hidden_size2, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss}")

Epoch 1, Loss: 1.1097946166992188


Epoch 2, Loss: 1.1086381673812866
Epoch 3, Loss: 1.1074906587600708
Epoch 4, Loss: 1.1063374280929565
Epoch 5, Loss: 1.1051570177078247
Epoch 6, Loss: 1.1039496660232544
Epoch 7, Loss: 1.102838158607483
Epoch 8, Loss: 1.1017757654190063
Epoch 9, Loss: 1.1006669998168945
Epoch 10, Loss: 1.0995147228240967
Epoch 11, Loss: 1.0983113050460815
Epoch 12, Loss: 1.0970594882965088
Epoch 13, Loss: 1.0957598686218262
Epoch 14, Loss: 1.0944137573242188
Epoch 15, Loss: 1.0930252075195312
Epoch 16, Loss: 1.0916029214859009
Epoch 17, Loss: 1.0901508331298828
Epoch 18, Loss: 1.0886669158935547
Epoch 19, Loss: 1.087151050567627
Epoch 20, Loss: 1.0856043100357056
Epoch 21, Loss: 1.0840296745300293
Epoch 22, Loss: 1.0824295282363892
Epoch 23, Loss: 1.0808048248291016
Epoch 24, Loss: 1.0791511535644531
Epoch 25, Loss: 1.0774600505828857
Epoch 26, Loss: 1.0757488012313843
Epoch 27, Loss: 1.074116826057434
Epoch 28, Loss: 1.0725889205932617
Epoch 29, Loss: 1.0711016654968262
Epoch 30, Loss: 1.0696390867233

In [ ]:
# Evaluate the model on the test set
model.eval()
with torch.no_grad():
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs, 1)
    accuracy = (predicted == y_test_tensor).sum().item() / len(y_test_tensor)
    print("Accuracy (WORD2VEC TERTIARY):", accuracy)

Accuracy (WORD2VEC TERTIARY): 0.61564


### **CONCAT METHOD**

#### **WORD2VEC BINARY**

In [ ]:
x = np.array([X_train_word2vec_binary[:10]])
# x = X_train_word2vec_binary
print(x.shape)

(1, 10, 300)


In [ ]:
def get_word2vec_features_concat(review_text):
  feature_vector = np.zeros((word2vec_model.vector_size,), dtype="float32")
  num_words = 0
  for i, word in enumerate(review_text):
    if word in word2vec_model.wv and i < 10:  # Limit to first 10 words
      feature_vector = np.add(feature_vector, word2vec_model.wv[word])
      num_words += 1
  if num_words != 0:
    feature_vector = np.divide(feature_vector, num_words)
  return feature_vector

# Represent each example with Word2Vec features
X_train_word2vec_concat = np.array([get_word2vec_features_concat(review) for review in tokenized_reviews])
print("X_train:")
print(X_train)
print('X_train_word2vec_concat:')
print(X_train_word2vec_concat)

X_train_word2vec_concat_binary = np.array([get_word2vec_features_concat(review) for review in tokenized_reviews_binary])
print("X_train_binary:")
print(X_train_binary)
print('X_train_word2vec_concat_binary:')
print(X_train_word2vec_concat_binary)

X_train:
75381     Good for printing \\"internal\\" documents tha...
65569                    These sticky notes are very small.
163473    Good quality for the price. Holds quite a bit ...
90518     two colors leaked, wouldn't work and one feels...
138866    As usual, when you get a bargain product, you ...
                                ...                        
119879    Tape cracks too easily even when unrolling tap...
103694    Ink cannot last 2 months. Original can use for...
131932    The pen is great I just wish it had normal col...
146867    IT was very easy to install and I got it up an...
121958    These cartridges are quite inexpensive compare...
Name: review_body, Length: 200000, dtype: object
X_train_word2vec_concat:
[[-0.75934184 -0.26071328 -0.1153479  ...  0.60092944 -0.6419013
  -0.50474095]
 [ 0.25292343 -0.63952917  0.024776   ... -0.22854434  0.8567886
  -0.76089936]
 [-0.06805354  0.4369895  -0.29323617 ...  0.2717982  -0.17278934
  -0.33616853]
 ...
 [-0.441258

In [ ]:
X_train_word2vec_concat.shape

(200000, 300)

In [ ]:
tokenized_reviews_test = [nltk.word_tokenize(review) for review in X_test]
tokenized_reviews_test_binary = [nltk.word_tokenize(review) for review in X_test_binary]

# Represent each example with Word2Vec features
X_test_word2vec_concat = np.array([get_word2vec_features_concat(review) for review in tokenized_reviews_test])
print("X_test:")
print(X_test)
print('X_test_word2vec_concat:')
print(X_test_word2vec_concat)

X_test_word2vec_concat_binary = np.array([get_word2vec_features_concat(review) for review in tokenized_reviews_test_binary])
print("X_test_binary:")
print(X_test_binary)
print('X_test_word2vec_concat_binary:')
print(X_test_word2vec_concat_binary)

X_test:
38683     I bought this printer for my Business, and it ...
64939     ...if it lasted. I'm glad I waited to write th...
3954      We purchased and installed this printer for a ...
120374    Everything that I have put into this machine, ...
172861    ordered as gift. recipient said it was great f...
                                ...                        
179545    the way you see those folders looking on the p...
222647                                  i'll buy again soon
171823    staff was helpful and tried very hard to make ...
135782                                       It's a fun pen
208380    This twine is thin but strong and is perfect f...
Name: review_body, Length: 50000, dtype: object
X_test_word2vec_concat:
[[-0.49564967  0.6754187   0.7017226  ...  0.5581657  -0.21785626
  -0.79002684]
 [-0.01967601  0.2927174   0.4976541  ...  0.09664098 -0.84558785
  -0.52363837]
 [-0.64603937  0.1970729   0.2222267  ...  0.5787168  -0.18215156
  -0.8590026 ]
 ...
 [-0.4400546

In [ ]:
X_test_word2vec_concat_binary.shape

(40000, 300)

In [ ]:
def get_pretrained_features_concat(review_text):
    feature_vector = np.zeros((pretrained_model.vector_size,), dtype="float32")
    num_words = 0
    for i, word in enumerate(review_text):
        if word in pretrained_model and i < 10: # limit to first 10
            feature_vector += pretrained_model[word]
            num_words += 1
    if num_words != 0:
        feature_vector /= num_words
    return feature_vector

# Represent each example with Word2Vec features
X_train_pretrained_concat = np.array([get_pretrained_features_concat(review) for review in tokenized_reviews])
print("X_train:")
print(X_train)
print('X_train_pretrained_concat:')
print(X_train_pretrained_concat)

X_train_pretrained_concat_binary = np.array([get_pretrained_features_concat(review) for review in tokenized_reviews_binary])
print("X_train_binary:")
print(X_train_binary)
print('X_train_pretrained_concat_binary:')
print(X_train_pretrained_concat_binary)

X_train:
75381     Good for printing \\"internal\\" documents tha...
65569                    These sticky notes are very small.
163473    Good quality for the price. Holds quite a bit ...
90518     two colors leaked, wouldn't work and one feels...
138866    As usual, when you get a bargain product, you ...
                                ...                        
119879    Tape cracks too easily even when unrolling tap...
103694    Ink cannot last 2 months. Original can use for...
131932    The pen is great I just wish it had normal col...
146867    IT was very easy to install and I got it up an...
121958    These cartridges are quite inexpensive compare...
Name: review_body, Length: 200000, dtype: object
X_train_pretrained_concat:
[[ 0.04692586  0.01741537 -0.02351888 ... -0.1127065  -0.02482096
   0.02710978]
 [-0.01839193  0.08548991 -0.02026367 ...  0.08823649  0.06197103
   0.03828939]
 [ 0.08663177  0.04251099 -0.06643677 ... -0.07038116  0.07638216
   0.03092194]
 ...
 [-0.01

In [ ]:
# Represent each example with Pretrained features
X_test_pretrained_concat = np.array([get_pretrained_features_concat(review) for review in tokenized_reviews_test])
print("X_test:")
print(X_test)
print('X_test_pretrained_concat:')
print(X_test_pretrained_concat)

X_test_pretrained_concat_binary = np.array([get_pretrained_features_concat(review) for review in tokenized_reviews_test_binary])
print("X_train_binary:")
print(X_test_binary)
print('X_test_pretrained_concat_binary:')
print(X_test_pretrained_concat_binary)

X_test:
38683     I bought this printer for my Business, and it ...
64939     ...if it lasted. I'm glad I waited to write th...
3954      We purchased and installed this printer for a ...
120374    Everything that I have put into this machine, ...
172861    ordered as gift. recipient said it was great f...
                                ...                        
179545    the way you see those folders looking on the p...
222647                                  i'll buy again soon
171823    staff was helpful and tried very hard to make ...
135782                                       It's a fun pen
208380    This twine is thin but strong and is perfect f...
Name: review_body, Length: 50000, dtype: object
X_test_pretrained_concat:
[[ 0.06853485 -0.01130891 -0.00944519 ... -0.00183487  0.0769043
  -0.07565308]
 [ 0.05960083  0.02112746  0.07992554 ... -0.03105164  0.07575226
  -0.0617981 ]
 [-0.03662981  0.06673758 -0.03023856 ... -0.04982648  0.01924133
  -0.15809849]
 ...
 [ 0.042026

In [ ]:
X_train_pretrained_concat_binary.shape

(160000, 300)

In [ ]:
# Convert the Word2Vec features to PyTorch tensors
X_train_tensor_binary = torch.tensor(X_train_word2vec_concat_binary, dtype=torch.float32)
y_train_tensor_binary = torch.tensor(y_train_binary.values - 1, dtype=torch.long)
X_test_tensor_binary = torch.tensor(X_test_word2vec_concat_binary, dtype=torch.float32)
y_test_tensor_binary = torch.tensor(y_test_binary.values - 1, dtype=torch.long)

# Define hyperparameters
input_size_binary = X_train_tensor_binary.shape[1]
hidden_size1 = 50
hidden_size2 = 10
output_size = 2  # Binary classification

In [ ]:
print(y_train_tensor_binary.shape)
print(X_train_tensor_binary.shape)

torch.Size([160000])
torch.Size([160000, 300])


In [ ]:
# Create DataLoader for training
train_dataset_binary = TensorDataset(X_train_tensor_binary, y_train_tensor_binary)
train_loader_binary = DataLoader(train_dataset_binary, batch_size=64, shuffle=True)

# Initialize the model, loss function, and optimizer
model = MLP(input_size_binary, hidden_size1, hidden_size2, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
#Training Loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor_binary)
    loss = criterion(outputs, y_train_tensor_binary)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss}")

Epoch 1, Loss: 0.6937284469604492
Epoch 2, Loss: 0.6905949711799622
Epoch 3, Loss: 0.6874700784683228
Epoch 4, Loss: 0.6843757033348083
Epoch 5, Loss: 0.6812653541564941
Epoch 6, Loss: 0.6780884265899658
Epoch 7, Loss: 0.6748496294021606
Epoch 8, Loss: 0.6715579032897949
Epoch 9, Loss: 0.6682333946228027
Epoch 10, Loss: 0.6648876667022705
Epoch 11, Loss: 0.6615219116210938
Epoch 12, Loss: 0.6581389307975769
Epoch 13, Loss: 0.6547418236732483
Epoch 14, Loss: 0.6513332724571228
Epoch 15, Loss: 0.647918701171875
Epoch 16, Loss: 0.6444942355155945
Epoch 17, Loss: 0.6410764455795288
Epoch 18, Loss: 0.6376779079437256
Epoch 19, Loss: 0.6343069076538086
Epoch 20, Loss: 0.6309748888015747
Epoch 21, Loss: 0.6276821494102478
Epoch 22, Loss: 0.6244305372238159
Epoch 23, Loss: 0.6212145090103149
Epoch 24, Loss: 0.6180214881896973
Epoch 25, Loss: 0.6148437261581421
Epoch 26, Loss: 0.6116703152656555
Epoch 27, Loss: 0.6085010170936584
Epoch 28, Loss: 0.6053378582000732
Epoch 29, Loss: 0.602188646793

In [ ]:
# Evaluate the model on the test set
model.eval()
with torch.no_grad():
    outputs = model(X_test_tensor_binary)
    _, predicted = torch.max(outputs, 1)
    accuracy = (predicted == y_test_tensor_binary).sum().item() / len(y_test_tensor_binary)
    print("Accuracy (WORD2VEC BINARY):", accuracy)

Accuracy (WORD2VEC BINARY): 0.76315


#### **PRE TRAINED BINARY**

In [ ]:
# Convert the Word2Vec features to PyTorch tensors
X_train_tensor_binary = torch.tensor(X_train_pretrained_concat_binary, dtype=torch.float32)
y_train_tensor_binary = torch.tensor(y_train_binary.values - 1, dtype=torch.long)
X_test_tensor_binary = torch.tensor(X_test_pretrained_concat_binary, dtype=torch.float32)
y_test_tensor_binary = torch.tensor(y_test_binary.values - 1, dtype=torch.long)

# Define hyperparameters
input_size_binary = X_train_tensor_binary.shape[1]
hidden_size1 = 50
hidden_size2 = 10
output_size = 2  # Binary classification

In [ ]:
# Create DataLoader for training
train_dataset_binary = TensorDataset(X_train_tensor_binary, y_train_tensor_binary)
train_loader_binary = DataLoader(train_dataset_binary, batch_size=64, shuffle=True)

# Initialize the model, loss function, and optimizer
model = MLP(input_size_binary, hidden_size1, hidden_size2, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
#Training Loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor_binary)
    loss = criterion(outputs, y_train_tensor_binary)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss}")

Epoch 1, Loss: 0.6937354207038879
Epoch 2, Loss: 0.6933442950248718
Epoch 3, Loss: 0.6929532885551453
Epoch 4, Loss: 0.6925543546676636
Epoch 5, Loss: 0.6921371817588806
Epoch 6, Loss: 0.6917063593864441
Epoch 7, Loss: 0.6912907958030701
Epoch 8, Loss: 0.6908850073814392
Epoch 9, Loss: 0.6904476284980774
Epoch 10, Loss: 0.6899695992469788
Epoch 11, Loss: 0.689461350440979
Epoch 12, Loss: 0.688934862613678
Epoch 13, Loss: 0.6883927583694458
Epoch 14, Loss: 0.6878263354301453
Epoch 15, Loss: 0.6872265338897705
Epoch 16, Loss: 0.6865859031677246
Epoch 17, Loss: 0.6859007477760315
Epoch 18, Loss: 0.6851728558540344
Epoch 19, Loss: 0.6844083666801453
Epoch 20, Loss: 0.6836192607879639
Epoch 21, Loss: 0.6828117966651917
Epoch 22, Loss: 0.6819773316383362
Epoch 23, Loss: 0.6811069846153259
Epoch 24, Loss: 0.6801964640617371
Epoch 25, Loss: 0.6792457103729248
Epoch 26, Loss: 0.6782565116882324
Epoch 27, Loss: 0.6772310137748718
Epoch 28, Loss: 0.6761707663536072
Epoch 29, Loss: 0.6750749945640

In [ ]:
# Evaluate the model on the test set
model.eval()
with torch.no_grad():
    outputs = model(X_test_tensor_binary)
    _, predicted = torch.max(outputs, 1)
    accuracy = (predicted == y_test_tensor_binary).sum().item() / len(y_test_tensor_binary)
    print("Accuracy (WORD2VEC BINARY):", accuracy)

Accuracy (WORD2VEC BINARY): 0.7259


#### **WORD2VEC TERTIARY**

In [ ]:
# Convert the Word2Vec features to PyTorch tensors
X_train_tensor = torch.tensor(X_train_word2vec_concat, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values - 1, dtype=torch.long)
X_test_tensor = torch.tensor(X_test_word2vec_concat, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values - 1, dtype=torch.long)

# Define hyperparameters
input_size = X_train_tensor.shape[1]
hidden_size1 = 50
hidden_size2 = 10
output_size = 3  # Tertiary classification

In [ ]:
print(y_train_tensor.shape)
print(X_train_tensor.shape)

torch.Size([200000])
torch.Size([200000, 300])


In [ ]:
# Create DataLoader for training
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Initialize the model, loss function, and optimizer
model = MLP(input_size, hidden_size1, hidden_size2, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
#Training Loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss}")

Epoch 1, Loss: 1.0911445617675781
Epoch 2, Loss: 1.0901182889938354
Epoch 3, Loss: 1.0887724161148071
Epoch 4, Loss: 1.0871578454971313
Epoch 5, Loss: 1.0853055715560913
Epoch 6, Loss: 1.083208680152893
Epoch 7, Loss: 1.0808581113815308
Epoch 8, Loss: 1.0782570838928223
Epoch 9, Loss: 1.0754159688949585
Epoch 10, Loss: 1.0723522901535034
Epoch 11, Loss: 1.0690921545028687
Epoch 12, Loss: 1.065659999847412
Epoch 13, Loss: 1.0620765686035156
Epoch 14, Loss: 1.0583627223968506
Epoch 15, Loss: 1.0545334815979004
Epoch 16, Loss: 1.0506020784378052
Epoch 17, Loss: 1.0465855598449707
Epoch 18, Loss: 1.0424928665161133
Epoch 19, Loss: 1.0383297204971313
Epoch 20, Loss: 1.0341094732284546
Epoch 21, Loss: 1.029843807220459
Epoch 22, Loss: 1.025551438331604
Epoch 23, Loss: 1.0212509632110596
Epoch 24, Loss: 1.0169700384140015
Epoch 25, Loss: 1.0127335786819458
Epoch 26, Loss: 1.008561372756958
Epoch 27, Loss: 1.004474401473999
Epoch 28, Loss: 1.000489592552185
Epoch 29, Loss: 0.9966208338737488
E

In [ ]:
# Evaluate the model on the test set
model.eval()
with torch.no_grad():
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs, 1)
    accuracy = (predicted == y_test_tensor).sum().item() / len(y_test_tensor)
    print("Accuracy (WORD2VEC TERTIARY):", accuracy)

Accuracy (WORD2VEC TERTIARY): 0.61114


#### **WORD2VEC TERTIARY**

In [ ]:
# Convert the Word2Vec features to PyTorch tensors
X_train_tensor = torch.tensor(X_train_pretrained_concat, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values - 1, dtype=torch.long)
X_test_tensor = torch.tensor(X_test_pretrained_concat, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values - 1, dtype=torch.long)

# Define hyperparameters
input_size = X_train_tensor.shape[1]
hidden_size1 = 50
hidden_size2 = 10
output_size = 3  # Tertiary classification

In [ ]:
# Create DataLoader for training
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Initialize the model, loss function, and optimizer
model = MLP(input_size, hidden_size1, hidden_size2, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
#Training Loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss}")

Epoch 1, Loss: 1.1017768383026123
Epoch 2, Loss: 1.1008857488632202
Epoch 3, Loss: 1.1000282764434814
Epoch 4, Loss: 1.0991886854171753
Epoch 5, Loss: 1.0984901189804077
Epoch 6, Loss: 1.0979965925216675
Epoch 7, Loss: 1.0976122617721558
Epoch 8, Loss: 1.0972579717636108
Epoch 9, Loss: 1.096907377243042
Epoch 10, Loss: 1.096556544303894
Epoch 11, Loss: 1.0961978435516357
Epoch 12, Loss: 1.0958251953125
Epoch 13, Loss: 1.095436692237854
Epoch 14, Loss: 1.0950311422348022
Epoch 15, Loss: 1.0946085453033447
Epoch 16, Loss: 1.0941686630249023
Epoch 17, Loss: 1.0937100648880005
Epoch 18, Loss: 1.0932316780090332
Epoch 19, Loss: 1.0927289724349976
Epoch 20, Loss: 1.0921945571899414
Epoch 21, Loss: 1.091617465019226
Epoch 22, Loss: 1.0909919738769531
Epoch 23, Loss: 1.0903249979019165
Epoch 24, Loss: 1.0896223783493042
Epoch 25, Loss: 1.0888864994049072
Epoch 26, Loss: 1.0881174802780151
Epoch 27, Loss: 1.0873159170150757
Epoch 28, Loss: 1.0864819288253784
Epoch 29, Loss: 1.0856133699417114
E

In [ ]:
# Evaluate the model on the test set
model.eval()
with torch.no_grad():
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs, 1)
    accuracy = (predicted == y_test_tensor).sum().item() / len(y_test_tensor)
    print("Accuracy (PRE-TRAINED TERTIARY):", accuracy)

Accuracy (PRE-TRAINED TERTIARY): 0.55782


# **Convolutional Neural Networks**

In [ ]:
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self, num_h1_nodes: int, num_h2_nodes: int, d: int, num_output_classes: int, dropout_rate: float):
        super().__init__()
        # self.conv1 = nn.Conv2d(3, 6, 5)
        # self.pool = nn.MaxPool2d(2, 2)
        # self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(d, num_h1_nodes)
        self.fc2 = nn.Linear(num_h1_nodes, num_h2_nodes)
        self.fc3 = nn.Linear(num_h2_nodes, num_output_classes)

    def forward(self, x):
        # x = self.pool(F.relu(self.conv1(x)))
        # x = self.pool(F.relu(self.conv2(x)))
        # x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

##### **WORD2VEC BINARY**

In [ ]:
# Convert the Word2Vec features to PyTorch tensors
X_train_tensor_binary = torch.tensor(X_train_word2vec_binary, dtype=torch.float32)
y_train_tensor_binary = torch.tensor(y_train_binary.values-1, dtype=torch.long)  # Assuming y_train_binary contains binary labels
X_test_tensor_binary = torch.tensor(X_test_word2vec_binary, dtype=torch.float32)
y_test_tensor_binary = torch.tensor(y_test_binary.values-1, dtype=torch.long)  # Convert Series to numpy array with .values attribute

dataset = TensorDataset(X_train_tensor_binary, y_train_tensor_binary)

# how many samples per batch to load
batch_size = 64

num_workers = 0

train_loader = DataLoader(dataset, batch_size=batch_size, num_workers=num_workers)

d_binary_embeddings = X_train_tensor_binary.shape[1]

# Define hyperparameters
num_h1_nodes = 50
num_h2_nodes = 10
num_output_classes = 2
dropout_rate = 0.2

In [ ]:
X_train_tensor_binary.shape[1]


300

In [ ]:
# Create the model and set it to training
model = CNN(num_h1_nodes, num_h2_nodes, d_binary_embeddings, num_output_classes, dropout_rate)

# model = model.train()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)


In [ ]:
# Training loop
for epoch in range(20):  # Adjust number of epochs
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 200 == 199:    # print every 200 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 200:.7f}')
            running_loss = 0.0
    # print(f"Binary Epoch {epoch+1}, Loss: {running_loss / 2000:.7f}")

[1,   200] loss: 0.6830409
[1,   400] loss: 0.6467744
[1,   600] loss: 0.5842589
[1,   800] loss: 0.5120663
[1,  1000] loss: 0.4535369
[1,  1200] loss: 0.4243788
[1,  1400] loss: 0.3942891
[1,  1600] loss: 0.3839101
[1,  1800] loss: 0.3802211
[1,  2000] loss: 0.3696721
[1,  2200] loss: 0.3613170
[1,  2400] loss: 0.3487234
[2,   200] loss: 0.3491343
[2,   400] loss: 0.3435925
[2,   600] loss: 0.3453553
[2,   800] loss: 0.3452436
[2,  1000] loss: 0.3421762
[2,  1200] loss: 0.3426039
[2,  1400] loss: 0.3299195
[2,  1600] loss: 0.3347692
[2,  1800] loss: 0.3360429
[2,  2000] loss: 0.3321724
[2,  2200] loss: 0.3313562
[2,  2400] loss: 0.3227181
[3,   200] loss: 0.3263650
[3,   400] loss: 0.3235906
[3,   600] loss: 0.3280013
[3,   800] loss: 0.3279585
[3,  1000] loss: 0.3274927
[3,  1200] loss: 0.3281851
[3,  1400] loss: 0.3161734
[3,  1600] loss: 0.3222112
[3,  1800] loss: 0.3229213
[3,  2000] loss: 0.3209815
[3,  2200] loss: 0.3201460
[3,  2400] loss: 0.3124270
[4,   200] loss: 0.3168920
[

In [ ]:
# Evaluate the model on the test set
model.eval()
with torch.no_grad():
    outputs = model(X_test_tensor_binary)
    _, predicted = torch.max(outputs, 1)
    accuracy = (predicted == y_test_tensor_binary).sum().item() / len(y_test_tensor_binary)
    print("Accuracy (WORD2VEC BINARY):", accuracy)

Accuracy (WORD2VEC BINARY): 0.881675


##### **WORD2VEC TERTIARY**

In [ ]:
# Convert the Word2Vec features to PyTorch tensors
X_train_tensor = torch.tensor(X_train_word2vec, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values-1, dtype=torch.long)  # Assuming y_train_binary contains binary labels
X_test_tensor = torch.tensor(X_test_word2vec, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_binary.values-1, dtype=torch.long)  # Convert Series to numpy array with .values attribute

dataset = TensorDataset(X_train_tensor, y_train_tensor)

# how many samples per batch to load
batch_size = 64

num_workers = 0

train_loader = DataLoader(dataset, batch_size=batch_size, num_workers=num_workers)

d_embeddings = X_train_tensor.shape[1]

# Define hyperparameters
num_h1_nodes = 50
num_h2_nodes = 10
num_output_classes = 3
dropout_rate = 0.2

In [ ]:
# Create the model and set it to training
model = CNN(num_h1_nodes, num_h2_nodes, d_binary_embeddings, num_output_classes, dropout_rate)

# model = model.train()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)


In [ ]:
# Training loop
for epoch in range(10):  # Adjust number of epochs
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 200 == 199:    # print every 200 mini-batches
        #     print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 200:.7f}')
            running_loss = 0.0
    print(f"Binary Epoch {epoch+1}, Loss: {running_loss / 2000:.7f}")

Binary Epoch 1, Loss: 0.0467122
Binary Epoch 2, Loss: 0.0439033
Binary Epoch 3, Loss: 0.0427430
Binary Epoch 4, Loss: 0.0420776
Binary Epoch 5, Loss: 0.0416304
Binary Epoch 6, Loss: 0.0412907
Binary Epoch 7, Loss: 0.0410043
Binary Epoch 8, Loss: 0.0407908
Binary Epoch 9, Loss: 0.0405953
Binary Epoch 10, Loss: 0.0404205


In [ ]:
# Evaluate the model on the test set
model.eval()
with torch.no_grad():
    outputs = model(X_test_tensor_binary)
    _, predicted = torch.max(outputs, 1)
    accuracy = (predicted == y_test_tensor_binary).sum().item() / len(y_test_tensor_binary)
    print("Accuracy (WORD2VEC BINARY):", accuracy)

Accuracy (WORD2VEC BINARY): 0.8243


##### **PRE-TRAINED BINARY**

In [ ]:
# Convert the Word2Vec features to PyTorch tensors
X_train_tensor_binary = torch.tensor(X_train_pretrained_binary, dtype=torch.float32)
y_train_tensor_binary = torch.tensor(y_train_binary.values-1, dtype=torch.long)  # Assuming y_train_binary contains binary labels
X_test_tensor_binary = torch.tensor(X_test_pretrained_binary, dtype=torch.float32)
y_test_tensor_binary = torch.tensor(y_test_binary.values-1, dtype=torch.long)  # Convert Series to numpy array with .values attribute

dataset = TensorDataset(X_train_tensor_binary, y_train_tensor_binary)

# how many samples per batch to load
batch_size = 64

num_workers = 0

train_loader = DataLoader(dataset, batch_size=batch_size, num_workers=num_workers)

d_binary_embeddings = X_train_tensor_binary.shape[1]

# Define hyperparameters
num_h1_nodes = 50
num_h2_nodes = 10
num_output_classes = 2
dropout_rate = 0.2

In [ ]:
# Create the model and set it to training
model = CNN(num_h1_nodes, num_h2_nodes, d_binary_embeddings, num_output_classes, dropout_rate)

# model = model.train()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)


In [ ]:
# Training loop
for epoch in range(20):  # Adjust number of epochs
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 200 == 199:    # print every 200 mini-batches
        #     print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 200:.7f}')
            running_loss = 0.0
    print(f"Binary Epoch {epoch+1}, Loss: {running_loss / 2000:.7f}")

Binary Epoch 1, Loss: 0.0306439
Binary Epoch 2, Loss: 0.0233038
Binary Epoch 3, Loss: 0.0210494
Binary Epoch 4, Loss: 0.0199838
Binary Epoch 5, Loss: 0.0193276
Binary Epoch 6, Loss: 0.0189169
Binary Epoch 7, Loss: 0.0186409
Binary Epoch 8, Loss: 0.0184492
Binary Epoch 9, Loss: 0.0183016
Binary Epoch 10, Loss: 0.0181784
Binary Epoch 11, Loss: 0.0180851
Binary Epoch 12, Loss: 0.0180102
Binary Epoch 13, Loss: 0.0179401
Binary Epoch 14, Loss: 0.0178757
Binary Epoch 15, Loss: 0.0178080
Binary Epoch 16, Loss: 0.0177409
Binary Epoch 17, Loss: 0.0176814
Binary Epoch 18, Loss: 0.0176245
Binary Epoch 19, Loss: 0.0175601
Binary Epoch 20, Loss: 0.0174817


In [ ]:
# Evaluate the model on the test set
model.eval()
with torch.no_grad():
    outputs = model(X_test_tensor_binary)
    _, predicted = torch.max(outputs, 1)
    accuracy = (predicted == y_test_tensor_binary).sum().item() / len(y_test_tensor_binary)
    print("Accuracy (WORD2VEC BINARY):", accuracy)

Accuracy (WORD2VEC BINARY): 0.8424


##### **PRE-TRAINED TERTIARY**

In [ ]:
# Convert the Word2Vec features to PyTorch tensors
X_train_tensor = torch.tensor(X_train_pretrained, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values-1, dtype=torch.long)  # Assuming y_train_binary contains binary labels
X_test_tensor = torch.tensor(X_test_pretrained, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_binary.values-1, dtype=torch.long)  # Convert Series to numpy array with .values attribute

dataset = TensorDataset(X_train_tensor, y_train_tensor)

# how many samples per batch to load
batch_size = 64

num_workers = 0

train_loader = DataLoader(dataset, batch_size=batch_size, num_workers=num_workers)

d_embeddings = X_train_tensor.shape[1]

# Define hyperparameters
num_h1_nodes = 50
num_h2_nodes = 10
num_output_classes = 3
dropout_rate = 0.2

In [ ]:
# Create the model and set it to training
model = CNN(num_h1_nodes, num_h2_nodes, d_binary_embeddings, num_output_classes, dropout_rate)

# model = model.train()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)


In [ ]:
# Training loop
for epoch in range(10):  # Adjust number of epochs
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 200 == 199:    # print every 200 mini-batches
        #     print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 200:.7f}')
            running_loss = 0.0
    print(f"Binary Epoch {epoch+1}, Loss: {running_loss / 2000:.7f}")

Binary Epoch 1, Loss: 0.0658773
Binary Epoch 2, Loss: 0.0657772
Binary Epoch 3, Loss: 0.0651549
Binary Epoch 4, Loss: 0.0605878
Binary Epoch 5, Loss: 0.0534047
Binary Epoch 6, Loss: 0.0509879
Binary Epoch 7, Loss: 0.0498970
Binary Epoch 8, Loss: 0.0492079
Binary Epoch 9, Loss: 0.0486316
Binary Epoch 10, Loss: 0.0482243


In [ ]:
# Evaluate the model on the test set
model.eval()
with torch.no_grad():
    outputs = model(X_test_tensor_binary)
    _, predicted = torch.max(outputs, 1)
    accuracy = (predicted == y_test_tensor_binary).sum().item() / len(y_test_tensor_binary)
    print("Accuracy (PRE-TRAINED TERTIARY):", accuracy)

Accuracy (PRE-TRAINED TERTIARY): 0.833475
